# Creacion Consolidado con audiencias

Comenzaremos el analisis de la data consolidando la info en un unico DataSet

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
from unicodedata import normalize
from pjud.data import transformdata
from pjud.data import cleandata

In [2]:
warnings.filterwarnings(action="ignore")
pd.set_option('display.max_columns', 100)

In [3]:
tqdm.pandas()

In [4]:
path_processed = "../data/processed/pjud"

In [5]:
df_audiencias = pd.read_feather(f"{path_processed}/processes_Audiencias.feather")
print(f"Existen : {len(df_audiencias)} audiencias")

Existen : 6169226 audiencias


In [6]:
df_audiencias.columns

Index(['level_0', 'index', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL',
       'RIT', 'TIPO CAUSA', 'TIPO DE AUDIENCIA',
       'FECHA PROGRAMACION AUDIENCIA', 'FECHA AUDIENCIA', 'DIAS AGENDAMIENTO',
       'DURACION AUDIENCIA (MIN)', 'TOTAL AUDIENCIAS'],
      dtype='object')

In [7]:
años = range(2015,2020)
audiencias_años = []
df_audiencias_años = df_audiencias.set_index("FECHA AUDIENCIA")

for año in años:
    #perido_inicial = '01-01-' + str(año)
    #periodo_final = '31-12-' + str(año)
    #mask_periodo = f"`FECHA AUDIENCIA` >='{perido_inicial}' and `FECHA AUDIENCIA` <= '{periodo_final}'"
    #mask_fecha = df_audiencias_años[año]
    total_año = len(df_audiencias_años[str(año)])
    #print(f"{mask_periodo} -> {total_año}")
    audiencias_años.append(total_año)

audiencias_años

[1198178, 1220384, 1229548, 1246377, 1274739]

Consolido ambas datas en un solo df

Unificar RIT y Tribunal en una sola columna para evitar mala interpretacion de causas

In [8]:
df_audiencias['TRIBUNAL-RIT'] = df_audiencias['COD. TRIBUNAL'].map(str) + "-" + df_audiencias['RIT'].map(str)

In [18]:
df_audiencias.sample(8)

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,TIPO DE AUDIENCIA,FECHA PROGRAMACION AUDIENCIA,FECHA AUDIENCIA,DIAS AGENDAMIENTO,DURACION AUDIENCIA (MIN),TOTAL AUDIENCIAS,TRIBUNAL-RIT,AÑO INGRESO CAUSA
5128206,5128206,177191,35,C.A. DE RANCAGUA,1063,JUZGADO DE GARANTIA SAN FERNANDO,4424-2014,Ordinaria,AUDIENCIA DE SOBRESEIMIENTO.,2015-03-18,2015-04-17,30,4,1,1063-4424-2014,2014
2935379,2935379,455870,90,C.A. DE SANTIAGO,1222,TERCER JUZGADO DE GARANTIA SANTIAGO,7902-2016,Ordinaria,AUDIENCIA DE PROCEDIMIENTO ABREVIADO.,2016-12-20,2017-01-11,22,2,1,1222-7902-2016,2016
5998962,5998962,1122817,90,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,1507-2015,Ordinaria,AUDIENCIA DE PREP. DE JUICIO ORAL SIMPLIF.,2015-06-11,2015-07-13,32,5,1,1226-1507-2015,2015
3549702,3549702,1123498,90,C.A. DE SANTIAGO,1222,TERCER JUZGADO DE GARANTIA SANTIAGO,676-2017,Ordinaria,AUDIENCIA DE ACUERDO REPARATORIO.,2017-06-21,2017-07-21,30,3,1,1222-676-2017,2017
3471907,3471907,1039348,55,C.A. DE VALDIVIA,226,JUZGADO DE LETRAS Y GARANTIA LA UNION,516-2017,Ordinaria,AUDIENCIA DE FORMAL. DE LA INVESTIGACISN.,2017-06-30,2017-07-24,24,12,1,226-516-2017,2017
1131221,1131221,1233411,90,C.A. DE SANTIAGO,1232,DECIMOTERCER JUZGADO DE GARANTIA SANTIAGO,6186-2017,Ordinaria,AUDIENCIA DE PREPARACION JUICIO ORAL.,2018-11-15,2018-12-17,32,8,1,1232-6186-2017,2017
744364,744364,815711,30,C.A. DE VALPARAISO,1046,JUZGADO DE GARANTIA VIÑA DEL MAR,13256-2017,Ordinaria,AUDIENCIA DE JUICIO ORAL SIMPLIFICADO,2018-01-23,2018-07-05,163,4,1,1046-13256-2017,2017
5751641,5751641,855386,35,C.A. DE RANCAGUA,1071,TRIBUNAL DE JUICIO ORAL EN LO PENAL RANCAGUA,132-2014,Ordinaria,AUDIENCIA DE JUICIO ORAL TOP,2015-06-08,2015-08-31,84,2,1,1071-132-2014,2014


Imputo algunos datos que si dispongo del dataset.

In [10]:
df_audiencias['AÑO INGRESO CAUSA'] = df_audiencias['RIT'].progress_apply(cleandata.obtiene_año)




100%|██████████| 6169226/6169226 [00:09<00:00, 632186.48it/s]


In [19]:
df_audiencias.columns

Index(['level_0', 'index', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL',
       'RIT', 'TIPO CAUSA', 'TIPO DE AUDIENCIA',
       'FECHA PROGRAMACION AUDIENCIA', 'FECHA AUDIENCIA', 'DIAS AGENDAMIENTO',
       'DURACION AUDIENCIA (MIN)', 'TOTAL AUDIENCIAS', 'TRIBUNAL-RIT',
       'AÑO INGRESO CAUSA'],
      dtype='object')

In [20]:
columnas_duplicadas = ['level_0', 'index']

df_audiencias.drop(columnas_duplicadas, axis='columns', inplace=True)



## Carga Data relacionada a Poblacion

In [21]:
df_poblacion = pd.read_feather(f"{path_processed}/processes_DataConsolidada_Poblacion_Jurisdiccion.feather")
print(f"{len(df_poblacion)} registros")

197 registros


In [22]:
df_poblacion

,index,REGION,CORTE,TRIBUNAL,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
0,0,REGION DE TARAPACA,C.A. DE IQUIQUE,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,330558,167793,162765,310065,20493,"[HUARA, CAMIÑA, COLCHANE, IQUIQUE, POZO ALMONT...",13,IQUIQUE,ORAL
1,1,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL CALAMA,177048,92417,84631,164011,13037,"[CALAMA, OLLAGUE, SAN PEDRO DE ATACAMA]",7,CALAMA,ORAL
2,2,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA,430486,222597,207889,407737,22749,"[TOCOPILLA, MARIA ELENA, MEJILLONES, SIERRA GO...",13,ANTOFAGASTA,ORAL
3,3,REGION DE ATACAMA,C.A. DE COPIAPO,TRIBUNAL DE JUICIO ORAL EN LO PENAL COPIAPO,286168,144420,141748,260520,25648,"[CHAÑARAL, DIEGO DE ALMAGRO, CALDERA, COPIAPO,...",9,COPIAPO,ORAL
4,4,REGION DE COQUIMBO,C.A. DE LA SERENA,TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA,496337,239495,256842,443484,52853,"[LA HIGUERA, VICUÑA, LA SERENA, COQUIMBO, ANDA...",10,LA SERENA,ORAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,192,REGION DE AISEN DEL GENERAL CARLOS IBAÑEZ DEL ...,C.A. DE COYHAIQUE,JUZGADO DE LETRAS Y GARANTIA COCHRANE,4638,2561,2077,2841,1797,"[COCHRANE, O'HIGGINS, TORTEL]",1,COCHRANE,LETRAS Y GARANTIA
193,193,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA NATALES,22686,11792,10894,19180,3506,"[NATALES, TORRES DEL PAINE]",2,NATALES,LETRAS Y GARANTIA
194,194,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA PORVENIR,8364,5266,3098,6062,2302,"[PORVENIR, PRIMAVERA, TIMAUKEL]",1,PORVENIR,LETRAS Y GARANTIA
195,195,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,2201,1321,880,1874,327,"[CABO DE HORNOS, ANTARTICA]",1,CABO DE HORNOS,LETRAS Y GARANTIA


In [23]:
df_fulldataaudiencias = pd.merge(df_audiencias, df_poblacion, how='left', on=['CORTE','TRIBUNAL'])

In [24]:
len(df_fulldataaudiencias)

6169226

In [27]:
df_fulldataaudiencias

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,TIPO DE AUDIENCIA,FECHA PROGRAMACION AUDIENCIA,FECHA AUDIENCIA,DIAS AGENDAMIENTO,DURACION AUDIENCIA (MIN),TOTAL AUDIENCIAS,TRIBUNAL-RIT,AÑO INGRESO CAUSA,index,REGION,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,AUDIENCIA DE JUICIO ORAL TOP,2018-04-03,2018-05-28,55,164,1,988-100-2018,2018,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,AUDIENCIA DE LECTURA DE SENTENCIA.,2018-06-01,2018-06-01,0,9,1,988-100-2018,2018,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,AUDIENCIA DE MEDIDAS CAUTELARES.,2018-06-19,2018-06-23,4,4,1,988-100-2018,2018,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,101-2018,Ordinaria,AUDIENCIA DE COMPARECENCIA JUDICIAL.,2018-04-03,2018-05-23,50,3,1,988-101-2018,2018,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,101-2018,Ordinaria,AUDIENCIA PRESENTACION VOL.IMPUTADO C/ORD. DETEN.,2018-05-24,2018-05-24,0,3,1,988-101-2018,2018,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6169221,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,85-2015,Ordinaria,AUDIENCIA DE LECTURA DE SENTENCIA.,2015-12-21,2015-12-24,3,3,1,1355-85-2015,2015,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
6169222,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,90-2013,Ordinaria,AUDIENCIA DE JUICIO ORAL TOP,2015-07-06,2015-08-12,37,197,1,1355-90-2013,2013,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
6169223,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,90-2013,Ordinaria,AUDIENCIA DE LECTURA DE SENTENCIA.,2015-08-12,2015-08-17,5,3,1,1355-90-2013,2013,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
6169224,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,92-2015,Ordinaria,AUDIENCIA DE COMPARECENCIA JUDICIAL.,2015-12-16,2015-12-21,5,7,1,1355-92-2015,2015,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL


In [29]:
df_fulldataaudiencias.columns

Index(['COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT', 'TIPO CAUSA',
       'TIPO DE AUDIENCIA', 'FECHA PROGRAMACION AUDIENCIA', 'FECHA AUDIENCIA',
       'DIAS AGENDAMIENTO', 'DURACION AUDIENCIA (MIN)', 'TOTAL AUDIENCIAS',
       'TRIBUNAL-RIT', 'AÑO INGRESO CAUSA', 'index', 'REGION', 'POBLACION',
       'HOMBRES', 'MUJERES', 'URBANO', 'RURAL', 'COMUNAS', 'JUECES', 'ASIENTO',
       'TIPO JUZGADO'],
      dtype='object')

VERIFICAR FILAS A ELIMINAR ... CAMBIARON ....

In [30]:
columnas_duplicadas = ['index']

df_fulldataaudiencias.drop(columnas_duplicadas, axis='columns', inplace=True)


In [31]:
#df_consolidado.drop(['MATERIA_x','FECHA TERMINO_x','AÑO TERMINO_x','TIPO CAUSA_y'], axis='columns', inplace=True)

In [33]:
df_fulldataaudiencias.sample(5)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,TIPO DE AUDIENCIA,FECHA PROGRAMACION AUDIENCIA,FECHA AUDIENCIA,DIAS AGENDAMIENTO,DURACION AUDIENCIA (MIN),TOTAL AUDIENCIAS,TRIBUNAL-RIT,AÑO INGRESO CAUSA,REGION,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
2475916,91,C.A. DE SAN MIGUEL,1236,JUZGADO DE GARANTIA PUENTE ALTO,17167-2018,Ordinaria,AUDIENCIA DE JUICIO ORAL SIMPLIFICADO,2019-07-25,2019-11-05,103,1,1,1236-17167-2018,2018,REGION METROPOLITANA DE SANTIAGO,612816,298437,314379,590972,21844,"[PUENTE ALTO, SAN JOSE DE MAIPO, PIRQUE]",8,PUENTE ALTO,GARANTIA
17875,11,C.A. DE IQUIQUE,989,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,758-2017,Ordinaria,AUDIENCIA DE LECTURA DE SENTENCIA.,2018-03-20,2018-03-23,3,1,1,989-758-2017,2017,REGION DE TARAPACA,330558,167793,162765,310065,20493,"[HUARA, CAMIÑA, COLCHANE, IQUIQUE, POZO ALMONT...",13,IQUIQUE,ORAL
4216057,90,C.A. DE SANTIAGO,1228,NOVENO JUZGADO DE GARANTIA SANTIAGO,10558-2015,Ordinaria,AUDIENCIA DE INCOMPETENCIA INHIBITORIA.,2016-05-03,2016-05-24,21,6,1,1228-10558-2015,2015,REGION METROPOLITANA DE SANTIAGO,602459,290423,312036,599026,3433,"[MAIPU, CERRILLOS]",9,SANTIAGO,GARANTIA
2615314,25,C.A. DE LA SERENA,930,JUZGADO DE GARANTIA COQUIMBO,5456-2016,Ordinaria,AUDIENCIA DE PROCEDIMIENTO SIMPLIFICADO.,2017-04-27,2017-06-05,39,1,1,930-5456-2016,2016,REGION DE COQUIMBO,227730,109872,117858,214550,13180,[COQUIMBO],3,COQUIMBO,GARANTIA
5339618,90,C.A. DE SANTIAGO,1221,SEGUNDO JUZGADO DE GARANTIA SANTIAGO,11941-2009,Ordinaria,AUDIENCIA DE SOBRESEIMIENTO.,2015-02-04,2015-04-29,84,3,1,1221-11941-2009,2009,REGION METROPOLITANA DE SANTIAGO,583187,286136,297051,582536,651,"[QUILICURA, HUECHURABA, RENCA, CONCHALI]",10,SANTIAGO,GARANTIA


Reordenando Nombres de las columnas ...

In [35]:
df_fulldataaudiencias.rename(columns = {'COD. CORTE':'cod_corte',
                                 'COD. TRIBUNAL':'cod_tribunal',
                                 'RIT':'rit',
                                 'CORTE':'corte',
                                 'TRIBUNAL':'tribunal',
                                 'TIPO CAUSA':'tipo_causa',
                                 'TIPO DE AUDIENCIA':'tipo_audiencia',
                                 'FECHA PROGRAMACION AUDIENCIA':'fecha_programacion_audiencia',
                                 'FECHA AUDIENCIA':'fecha_audiencia',
                                 'DIAS AGENDAMIENTO':'dias_agendamiento',
                                 'DURACION AUDIENCIA (MIN)':'duracion_audiencia_minutos',
                                 'TOTAL AUDIENCIAS':'total_audiencias',
                                 'TRIBUNAL-RIT':'tribunal_rit',
                                 'AÑO INGRESO CAUSA':'año_ingreso',
                                 'REGION':'region',
                                 'POBLACION':'poblacion',
                                 'HOMBRES':'hombres',
                                 'MUJERES':'mujeres',
                                 'URBANO':'urbano',
                                 'RURAL':'rural',
                                 'COMUNAS':'comunas',
                                 'JUECES':'dotacion_jueces',
                                 'ASIENTO':'asiento',
                                 'TIPO JUZGADO':'tipo_juzgado'
                                 },inplace = True)

In [36]:
df_fulldataaudiencias.columns

Index(['cod_corte', 'corte', 'cod_tribunal', 'tribunal', 'rit', 'tipo_causa',
       'tipo_audiencia', 'fecha_programacion_audiencia', 'fecha_audiencia',
       'dias_agendamiento', 'duracion_audiencia_minutos', 'total_audiencias',
       'tribunal_rit', 'año_ingreso', 'region', 'poblacion', 'hombres',
       'mujeres', 'urbano', 'rural', 'comunas', 'dotacion_jueces', 'asiento',
       'tipo_juzgado'],
      dtype='object')

In [37]:
df_fulldataaudiencias = df_fulldataaudiencias[['region','cod_corte','corte','tribunal_rit','cod_tribunal','rit','tribunal','tipo_juzgado','dotacion_jueces','tipo_causa','año_ingreso',
                           'tipo_audiencia','fecha_programacion_audiencia','fecha_audiencia','dias_agendamiento','duracion_audiencia_minutos','total_audiencias',
                           'asiento','comunas','poblacion','hombres','mujeres','urbano','rural']]

In [38]:
df_fulldataaudiencias.sample(5)

,region,cod_corte,corte,tribunal_rit,cod_tribunal,rit,tribunal,tipo_juzgado,dotacion_jueces,tipo_causa,año_ingreso,tipo_audiencia,fecha_programacion_audiencia,fecha_audiencia,dias_agendamiento,duracion_audiencia_minutos,total_audiencias,asiento,comunas,poblacion,hombres,mujeres,urbano,rural
4430018,REGION DE ATACAMA,20,C.A. DE COPIAPO,956-1453-2016,956,1453-2016,JUZGADO DE GARANTIA VALLENAR,GARANTIA,2,Ordinaria,2016,AUDIENCIA DE CONTROL DE LA DETENCION.,2016-08-24,2016-08-24,0,5,1,VALLENAR,"[VALLENAR, ALTO DEL CARMEN]",57216,28347,28869,46019,11197
1188546,REGION METROPOLITANA DE SANTIAGO,91,C.A. DE SAN MIGUEL,1234-4137-2018,1234,4137-2018,DECIMOQUINTO JUZGADO DE GARANTIA SANTIAGO,GARANTIA,7,Ordinaria,2018,AUDIENCIA DE PROCEDIMIENTO ABREVIADO.,2018-11-08,2018-11-20,12,25,1,SANTIAGO,"[SAN RAMON, LA PINTANA]",260235,127917,132318,260235,0
2662031,REGION DE VALPARAISO,30,C.A. DE VALPARAISO,1050-772-2016,1050,772-2016,JUZGADO DE GARANTIA CASABLANCA,GARANTIA,1,Ordinaria,2016,AUDIENCIA DE ACUERDO REPARATORIO.,2017-04-03,2017-05-03,30,4,1,CASABLANCA,[CASABLANCA],26867,13346,13521,18002,8865
3986053,REGION DE ÑUBLE,45,C.A. DE CHILLAN,1072-1559-2016,1072,1559-2016,JUZGADO DE GARANTIA CHILLAN,GARANTIA,4,Ordinaria,2016,AUDIENCIA DE MEDIDAS CAUTELARES.,2016-06-22,2016-06-29,7,35,1,CHILLAN,"[SAN NICOLAS, CHILLAN, COIHUECO, PINTO, CHILLA...",264957,126658,138299,215499,49458
1536806,REGION DEL BIO BIO,46,C.A. DE CONCEPCION,1080-2159-2019,1080,2159-2019,JUZGADO DE GARANTIA TALCAHUANO,GARANTIA,4,Ordinaria,2019,AUDIENCIA DE CONTROL DE LA DETENCION.,2019-05-24,2019-05-23,0,9,1,TALCAHUANO,"[TALCAHUANO, HUALPEN]",243522,116481,127041,241305,2217


In [40]:
# Directorio donde se guardaran archivos feather
df_fulldataaudiencias.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_fulldataaudiencias.to_feather(f'{path_processed}/consolidated_FULLDATA_AUDIENCIAS.feather')